<a href="https://colab.research.google.com/github/SfurtiR/Natural-Language-Processing/blob/main/Sentiment_Analysis_NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Sentiment Analysis NLP project***

We will:
*   Preprocess text using the pipeline.
*   Use TextBlob for sentiment analysis.
*   Use scikit-learn to train a simple model for sentiment classification.


**Step 1 :Install and Import Required Libraries**





In [ ]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.3 MB/s eta 0:00:00


In [ ]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.9 MB/s eta 0:00:00


In [ ]:
import re
import string
import emoji
import contractions
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


**Step 2 :Preprocessing Function for Sentiment Analysis**


In [ ]:
# Download necessary NLTK datasets
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")


In [ ]:
def preprocess_text(text):
    """Function to clean and preprocess text data for sentiment analysis."""

    # 1. Lowercasing
    text = text.lower()

    # 2. Expand contractions (e.g., "don't" → "do not")
    text = contractions.fix(text)

    # 3. Remove Emojis
    text = emoji.demojize(text, delimiters=(" ", " "))

    # 4. Remove Punctuation & Special Characters
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Tokenization
    words = word_tokenize(text)

    # 6. Remove Stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # 7. Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # 8. Spelling Correction
    words = [str(TextBlob(word).correct()) for word in words]

    return " ".join(words)  # Return processed text as a string


**Step 3: Load Sentiment Data**



We’ll use a sample dataset with text and sentiment labels (0 for negative, 1 for positive).


In [ ]:
# Example dataset (real projects use larger datasets)
data = {
    "text": [
        "I love this product! It's amazing 😊",
        "This is the worst experience ever. I hate it!",
        "I'm so happy with my purchase. It's perfect!",
        "The quality is terrible and I'm very disappointed.",
        "Absolutely wonderful! Will buy again.",
        "Not worth the money. Poor quality.",
        "Great customer service and fast delivery!",
        "This was a waste of my time and money."
    ],
    "sentiment": [1, 0, 1, 0, 1, 0, 1, 0]  # 1 = Positive, 0 = Negative
}

df = pd.DataFrame(data)

# Apply the preprocessing function
df['clean_text'] = df['text'].apply(preprocess_text)

# Display the first few rows
print(df.head())


                                                text  sentiment  \
0                I love this product! It's amazing 😊          1   
1      This is the worst experience ever. I hate it!          0   
2       I'm so happy with my purchase. It's perfect!          1   
3  The quality is terrible and I'm very disappoin...          0   
4              Absolutely wonderful! Will buy again.          1   

                                        clean_text  
0  love product amazing smilingfacewithsmilingeyes  
1                       worst experience ever hate  
2                           happy purchase perfect  
3                    quality terrible disappointed  
4                         absolutely wonderful buy  


**Step 4: Convert Text Data into Numerical Features**

We use TF-IDF (Term Frequency-Inverse Document Frequency) to convert text into a numerical format for training.

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['sentiment'], test_size=0.2, random_state=42)

# Convert text into numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


**Step 5: Train a Machine Learning Model**

We’ll use Naive Bayes (MultinomialNB), which is great for text classification tasks.


In [ ]:
# Train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
y_pred


Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.50      0.25      0.33         2
weighted avg       1.00      0.50      0.67         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([1, 0])

**Step 6: Test the Model on New Data**

In [ ]:
def predict_sentiment(text):
    """Function to predict sentiment of a given text"""
    clean_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([clean_text])
    prediction = model.predict(text_tfidf)[0]
    return "Positive" if prediction == 1 else "Negative"

# Test with new examples
test_texts = [
    "I absolutely love this! It's fantastic.",
    "Worst product ever. I regret buying it!",
    "Not bad, but could be better.",
    "The customer service was very helpful!"
]

for text in test_texts:
    print(f"Text: {text} --> Sentiment: {predict_sentiment(text)}")


Text: I absolutely love this! It's fantastic. --> Sentiment: Positive
Text: Worst product ever. I regret buying it! --> Sentiment: Positive
Text: Not bad, but could be better. --> Sentiment: Positive
Text: The customer service was very helpful! --> Sentiment: Positive
